In [25]:
import mne
import numpy as np
import pandas as pd
import autoreject
import os

In [26]:
def clean_montage(montage_path):
    '''
    This function takes a path to a montage file and returns a montage object
    rescales the montage coordinates and takes care of any NaNs or missing channels
    '''
    montage_data = pd.read_csv(montage_path, sep='\t', header=0, engine='python')
    montage_data = montage_data.set_index('name', drop=True)
    montage_data.loc['Resp'] = [0,0,0]
    montage_data.replace(np.nan, 0, inplace=True)
    #montage_data.dropna(inplace=True)
    scale = 0.095  #scale based on head radius (value of 1 means equal to head radius)
    montage_data.loc[:, ['x','y','z']] *= scale #rescale coordinates
    mapping = montage_data.T.to_dict('list') #create a mapping from channel name to coordinates
    montage = mne.channels.make_dig_montage(ch_pos=mapping,coord_frame='head') #prepare montage from mapping
    return montage


In [27]:
def preprocess(raw_data_path, montage_path):
    '''
    utility function
    raw_data_path: path to subject's folder containing raw data
    montage_path: path to subject's folder containing montage data
    '''
    rd= mne.io.read_raw_eeglab(raw_data_path, preload=True,verbose=False) #load data
    acti_cap_mon=clean_montage(montage_path) #load montage
    rd.set_montage(acti_cap_mon, on_missing='ignore') #set montage
    #raw_plot=rd.plot()
    chans_to_remove = ['FT9','FT10','TP9','TP10']
    rd.drop_channels([chan for chan in chans_to_remove if chan in rd.ch_names]) #remove unreliable channels
    rd.drop_channels([chan for chan in rd.ch_names if chan not in acti_cap_mon.ch_names]) #remove channels not in montage
    rd.set_eeg_reference(ref_channels='average') #rereference to average
    new_sampling_freq = 256 #new sampling frequency
    rd.resample(new_sampling_freq)  #resample

    rd= rd.copy().filter(l_freq=0.1, h_freq=None) #highpass filter

    events,event_dict = mne.events_from_annotations(rd)  #{'S  1': 1, 'S  2': 2, 'S  3': 3, 'boundary': 4}
    epochs= mne.Epochs(rd, events, tmin=-1, tmax=2.5, event_id=event_dict, preload=True) #epoching

    epochs.info['bads'] = ['Cz'] #exlude Cz channel from ICA
    ica = mne.preprocessing.ICA(n_components=15, random_state=50, max_iter=800) #perform ICA
    ica.fit(epochs)  #fit ICA
    ica.apply(epochs) #apply ICA
    #after_ica=rd.plot()
    
    del rd #delete raw data to save memory; we only need epochs now
   
    ar= autoreject.AutoReject(n_interpolate=[1,2,3,4],random_state=11,n_jobs=1,verbose=True) #perform autoreject to remove bad epochs
    ar.fit(epochs[:10])
    epochs_arr, reject_log = ar.transform(epochs, return_log=True)
    epochs_arr.interpolate_bads() #interpolate bad channels exlcuding Cz
    try:
        epochs_arr.info['bads'].remove('Cz') #remove Cz channel from bad channels
    except:
        pass

    return epochs_arr

In [28]:
def get_subject_folders(eeg_path):
    '''
    utility function
    Returns a list of all subject folders in the given path
    '''
    folders = []
    for root, dirs, _ in os.walk(eeg_path):
        folders.extend([os.path.join(root, d) for d in dirs]) 
        break # only top level subfolders required 
    return folders

In [29]:

def batch_preprocess(input_data_path,output_data_path):
    '''
    input_data_path: path to the folder containing the subject folders.
    output_data_path: path to the folder where the preprocessed data will be saved.
    '''
    if not os.path.exists(output_data_path):
        os.makedirs(output_data_path)
    
    subjects = get_subject_folders(input_data_path)

    for subject in subjects:
        print("\npreproccessing subject: " + subject + "...")
        montage_path = os.path.join(subject,'eeg',subject[-7:]+'_task-Oddball_electrodes.tsv')
        out = preprocess(os.path.join(subject,'eeg',subject[-7:]+'_task-Oddball_eeg.set'),montage_path)
        out_path = os.path.join(output_data_path, subject[-7:])
        os.makedirs(out_path)
        out.save(os.path.join(out_path, subject[-7:] + "-epo.fif"))
        del out
        print("preproccessed subject: " + subject + "!\n")

In [30]:
input_data_path = r'C:\Users\EdgardOrdoñez\OneDrive - AltiaTek\Desktop\pxs'  # Debe ser la ruta al directorio que contiene carpetas de sujetos
output_data_path = r'C:\Users\EdgardOrdoñez\OneDrive - AltiaTek\Desktop\preprocessed'
batch_preprocess(input_data_path, output_data_path)


preproccessing subject: C:\Users\EdgardOrdoñez\OneDrive - AltiaTek\Desktop\pxs\sub-001...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


C:\Users\EdgardOrdoñez\AppData\Local\Temp\ipykernel_23548\3000635427.py:7: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  rd= mne.io.read_raw_eeglab(raw_data_path, preload=True,verbose=False) #load data


Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Filter length: 8449 samples (33.004 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Used Annotations descriptions: ['S  1', 'S  2', 'S  3', 'boundary']
Not setting metadata
709 matching events found
Setting baseline interval to [-1.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 709 events and 897 original time points ...
1 bad epochs dropped
Fitting ICA to data using 58 channels (please be patient, this may take a while)


...

100%|██████████| Creating augmented epochs : 58/58 [00:01<00:00,   41.28it/s]
100%|██████████| Computing thresholds ... : 58/58 [00:08<00:00,    6.65it/s]



100%|██████████| Repairing epochs : 10/10 [00:00<00:00,  268.67it/s]







100%|██████████| Repairing epochs : 10/10 [00:00<00:00,   54.85it/s]






















100%|██████████| Fold : 10/10 [00:01<00:00,    9.15it/s]






100%|██████████| Repairing epochs : 10/10 [00:00<00:00,   73.24it/s]






















100%|██████████| Fold : 10/10 [00:00<00:00,   12.26it/s]









100%|██████████| Repairing epochs : 10/10 [00:00<00:00,   38.30it/s]






















100%|██████████| Fold : 10/10 [00:01<00:00,    6.45it/s]











100%|██████████| Repairing epochs : 10/10 [00:00<00:00,   29.73it/s]






















100%|██████████| Fold : 10/10 [00:01<00:00,    9.02it/s]
100%|██████████| n_interp : 4/4 [00:05<00:00,    1.43s/it]
c:\Users\EdgardOrdoñez\AppData\Local\Programs\Python\Python312\Lib\site-packages\autoreje





Estimated consensus=0.40 and n_interpolate=4







































































































































































































































































































































































































































































































































































































100%|██████████| Repairing epochs : 708/708 [00:14<00:00,   49.65it/s]


Dropped 377 epochs: 18, 20, 25, 26, 48, 57, 58, 59, 60, 61, 62, 63, 64, 65, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 84, 85, 86, 89, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 150, 151, 152, 153, 159, 160, 161, 162, 216, 217, 218, 234, 235, 236, 240, 241, 242, 243, 249, 250, 251, 252, 253, 254, 258, 259, 260, 261, 262, 263, 264, 265, 266, 279, 280, 281, 282, 283, 284, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 324, 325, 326, 330, 331, 332, 333, 334, 335, 336, 337, 338, 342, 343, 344, 345, 347, 351, 352, 353, 354, 355, 356, 357, 358, 359, 361, 362, 369, 370, 371, 378, 379, 380, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 411, 412, 413, 414, 415, 416, 417, 418, 419, 425, 426, 427, 428, 42

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\EdgardOrdoñez\\OneDrive - AltiaTek\\Desktop\\preprocessed\\sub-001'